### Required Modules

In [45]:
!pip install lifetimes
!pip install pandas_gbq

  Using cached pandas_gbq-0.17.4-py2.py3-none-any.whl (25 kB)
  Using cached pydata_google_auth-1.4.0-py2.py3-none-any.whl (14 kB)


In [61]:
from lifetimes import ParetoNBDFitter,GammaGammaFitter
import pandas as pd
import numpy as np
import math
from google.cloud import bigquery
import datetime
import time
import gc
import pandas_gbq

In [47]:
gc.collect()

123

## Config

In [48]:
PROJECT_NAME='cdp-dev-bdfa'
RFM_TABLE='harmonized_layer.RFM'
PREDICTIONS_TABLE='harmonized_layer.CLTV_Model_Predictions'
CHECKPOINTS_TABLE='harmonized_layer.CLTV_Model_Checkpoints'
CLTV_PERIOD=12 #months
CLTV_EXPECTED_TRIPS_PERIOD=365 #days

client = bigquery.Client(project=PROJECT_NAME)

## Loading the data from big query

In [49]:
sql_query = (f"SELECT frequency,recency,T,monetary_value,Count(customer_id) as segments FROM `{RFM_TABLE}` GROUP BY frequency,recency,T,monetary_value")
df = client.query(sql_query).to_dataframe().set_index('segments')

In [50]:
df.head()

,frequency,recency,T,monetary_value
segments,,,,
10,0,0.0,38.86,0.0
15,0,0.0,38.71,0.0
12,0,0.0,38.57,0.0
13,0,0.0,38.43,0.0
14,0,0.0,38.29,0.0


## Building Pareto Model

In [51]:
pnf = ParetoNBDFitter(penalizer_coef=0.0)
pnf.fit(df['frequency'], df['recency'], df['T'],weights=df.index.tolist())

<lifetimes.ParetoNBDFitter: fitted with 1030 subjects, alpha: 10.58, beta: 11.67, r: 0.55, s: 0.61>

In [52]:
simulated_data = pnf.generate_new_data(size=pnf.data.shape[0])
rmse=np.sqrt(np.mean(np.square(df['frequency'] - simulated_data['frequency'])))
pnf_fitter_param=pnf.params_

## Building Gamma Gamma Model

#### Note-:
To calculate the CLV, we need to know the returning customer summary, which is required by gammafitter

In [53]:
returning_customers_summary = df[df['frequency']>0]

##### Note-: 
The Gamma-Gamma submodel, in fact, assumes that there is no relationship between the monetary value and the purchase frequency. In practice we need to check whether the Pearson correlation between the two vectors is close to 0 in order to use this model.

In [54]:
corr_=returning_customers_summary[['monetary_value', 'frequency']].corr()

if not corr_.loc['monetary_value','frequency']<0.15:
  print("Please rectify,Its above threshold. Here gamma-gamma model assumes that no relationship between monetary value and purchase frequency")

In [55]:
ggf=GammaGammaFitter(penalizer_coef = 0)
ggf.fit(returning_customers_summary['frequency'],
        returning_customers_summary['monetary_value'])
ggf_fitter_param=ggf.params_

### Generating KPI's For next 12 months

###### Batch predictions, Processing 10M records at a time

In [56]:
total_rows=client.query(f"SELECT max(customer_id) as total_row  FROM `{RFM_TABLE}`").to_dataframe().values[0][0]
BATCH_SIZE=1000000
batches={}
for index,offset  in enumerate(range(0,total_rows,BATCH_SIZE)):
    batches[f"batch{index}"]={'limit':BATCH_SIZE,'offset':offset}

In [57]:
def get_data_batch(client: bigquery.Client,limit:int, offset:int) -> pd.DataFrame:
    
    print("Fetching Data")
    
    sql_query = (f"select * from {RFM_TABLE} limit {limit} offset {offset}")
    
    df = client.query(sql_query).to_dataframe().set_index('customer_id')
    
    return df


In [58]:
def compute_kpi(data:pd.DataFrame) -> pd.DataFrame:

    #Generating CLTV Values
    cltv_values=ggf.customer_lifetime_value(
                    pnf, #the model to use to predict the number of future transactions
                    data['frequency'],
                    data['recency'],
                    data['T'],
                    data['monetary_value'],
                    time=CLTV_PERIOD, # months
                    discount_rate=0.00) 
    cltv_values.name="CUST_LIFETIME_VAL_12MO"

    """Generating future expected number of purchase, Here t is in periods which is measured in dates"""
    cltv_future_purchase=pnf.conditional_expected_number_of_purchases_up_to_time(CLTV_EXPECTED_TRIPS_PERIOD,data['frequency'], data['recency'], data['T'])
    cltv_future_purchase.name="FREQ_FUTURE_12MO_PURCHASE"


    """Generating conditional probabilty of being alive,It calculates the probability of a customer being currently alive or not"""
    cltv_prob_alive=pnf.conditional_probability_alive(data['frequency'], data['recency'], data['T'])
    cltv_prob_alive.name="PROB_ALIVE"

    return pd.concat([cltv_values,cltv_prob_alive,cltv_future_purchase],axis=1)


In [59]:
def save_metric_bq(data:pd.DataFrame,method:str):
    print("Saving computed KPI's to bigquery")

    table_id=f"{PREDICTIONS_TABLE}"
    pandas_gbq.to_gbq(data.round(2).reset_index(), table_id, project_id=PROJECT_NAME,if_exists=method)
    
def batch_prediction():
    for key,value in batches.items():

        #Loading the data
        df=get_data_batch(client,value['limit'],value['offset'])  

        #computing KPI's for next 12 months
        df=compute_kpi(df)

        #adding timestamp to the dataframe
        f = '%Y-%m-%d %H:%M:%S'
        timestamp_now = datetime.datetime.now().strftime(f)
        df['timestamp']=timestamp_now

        # This if block denotes, whether we want to replace the existing table if exist or append
        if value['offset']==0:
            save_metric_bq(df,'replace')
        else:
            save_metric_bq(df,'append')
    print("Batch Prediction Completed")

In [62]:
batch_prediction()

Fetching Data
Saving computed KPI's to bigquery


100%|██████████| 1/1 [00:00<00:00, 2186.81it/s]

Batch Prediction Completed
